# **Programming Assessment \#4**

Names: Alyanna Abalos, Loben Tipan

More information on the assessment is found in our Canvas course.

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [3]:
# Read the content of the files
with open('count_1edit.txt', 'r') as file:
    data_count_1edit = file.read()

with open('spell_errors.txt', 'r') as file:
    data_spell_errors = file.read()


Failed to fetch data.


# **Noisy Channel Model Implementation**

*Again, you don't have to follow this directly, but consider placing your implementation of the model in the code block below.*